# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

# 1) Camera calibration using chessboard images

# 2) Apply a distortion correction to raw images

# 3) Use color transforms, gradients, etc., to create a thresholded binary image.

# 4) Apply a perspective transform to rectify binary image ("birds-eye view")

# 5) Detect lane pixels and fit to find the lane boundary

# 6) Determine the curvature of the lane and vehicle position with respect to center.

# 7) Warp the detected lane boundaries back onto the original image.

# 8) Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.